# Bước 3: Apror Modelng for Assocaton Rules

Notebook này sử dụng ma trận `basket_bool` (được chuẩn bị ở bước 02) để:

- Kha thác **tập mục phổ bến** (frequent temsets) bằng thuật toán **Apror**
- Snh **luật kết hợp** (assocaton rules) vớ các chỉ số: `support`, `confdence`, `lft`
- Lọc luật theo các ngưỡng do ngườ dùng cấu hình
- Trực quan hoá một số nhóm luật têu bểu phục vụ storytellng & phân tích knh doanh
- Lưu kết quả luật đã lọc ra fle `.csv` để sử dụng trong báo cáo, dashboard hoặc các bước phân tích tếp theo

Notebook được thết kế theo kểu *parameterzed* để dễ dàng tích hợp vớ **papermll**, cho phép chạy tự động vớ các ngưỡng support/confdence khác nhau mà không cần sửa code.


## Set up

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Biểu đồ tương tác HTML
import plotly.express as px

from apriori_library import FPGrowthRulesMiner  # class trong library của bạn


## Thiết lập style vẽ biểu đồ 

In [2]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12


## Tải basket_bool

In [3]:
# PARAMETERS (FP-Growth)

BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"


In [4]:
# Đọc ma trận basket_bool từ bước 2
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)

print("=== Thông tin basket_bool ===")
print(f"- Số hoá đơn (rows): {basket_bool.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket_bool.shape[1]:,}")
print(f"- Tỷ lệ ô = 1 (có mua): {basket_bool.values.mean():.4f}")

basket_bool.head()


=== Thông tin basket_bool ===
- Số hoá đơn (rows): 18,021
- Số sản phẩm (columns): 4,007
- Tỷ lệ ô = 1 (có mua): 0.0066


,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Khai thác tập phổ biến bằng thuật toán Apriori

In [5]:
# PARAMETERS (Lab 2 - dùng chung cho Apriori & FP-Growth)

MIN_SUPPORT = 0.02
MAX_LEN = 3

METRIC = "lift"
MIN_THRESHOLD = 1.0


In [6]:
# Khởi tạo Apriori miner
miner_fp = FPGrowthRulesMiner(basket_bool)

start_time = time.time()
frequent_itemsets_fp = miner_fp.mine_frequent_itemsets(
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN
)
elapsed_time_fp = time.time() - start_time


print("=== Kết quả khai thác tập mục phổ biến (FP-Growth) ===")
print(f"- Thời gian chạy: {elapsed_time_fp:.2f} giây")
print(f"- Số tập mục phổ biến thu được: {len(frequent_itemsets_fp):,}")

frequent_itemsets_fp.head(10)


=== Kết quả khai thác tập mục phổ biến (FP-Growth) ===
- Thời gian chạy: 7.18 giây
- Số tập mục phổ biến thu được: 400


,support,itemsets
0,0.119971,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.024860,(KNITTED UNION FLAG HOT WATER BOTTLE)
2,0.022807,(RED WOOLLY HOTTIE WHITE HEART.)
3,0.026414,(HAND WARMER UNION JACK)
4,0.076078,(ASSORTED COLOUR BIRD ORNAMENT)
5,0.041507,(HOME BUILDING BLOCK WORD)
6,0.033683,(LOVE BUILDING BLOCK WORD)
7,0.031186,(DOORMAT NEW ENGLAND)
8,0.022918,(FELTCRAFT PRINCESS CHARLOTTE DOLL)
9,0.020809,(POPPY'S PLAYHOUSE KITCHEN)


## Sinh luật kết hợp từ tập mục phổ biến

In [7]:
# Sinh luật FP-Growth
rules_fp = miner_fp.generate_rules(
    metric=METRIC,
    min_threshold=MIN_THRESHOLD
)

print("Số luật FP-Growth:", rules_fp.shape[0])
rules_fp.head()


Số luật FP-Growth: 218


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(JAM MAKING SET WITH JARS),(JAM MAKING SET PRINTED),0.055435,0.055935,0.023306,0.420420,7.516266,1.0,0.020205,1.628879,0.917836,0.264650,0.386081,0.418544
1,(JAM MAKING SET PRINTED),(JAM MAKING SET WITH JARS),0.055935,0.055435,0.023306,0.416667,7.516266,1.0,0.020205,1.619254,0.918321,0.264650,0.382432,0.418544
2,(WOODEN PICTURE FRAME WHITE FINISH),(WHITE HANGING HEART T-LIGHT HOLDER),0.059708,0.119971,0.022418,0.375465,3.129625,1.0,0.015255,1.409094,0.723683,0.142555,0.290324,0.281164
3,(WHITE HANGING HEART T-LIGHT HOLDER),(WOODEN PICTURE FRAME WHITE FINISH),0.119971,0.059708,0.022418,0.186864,3.129625,1.0,0.015255,1.156377,0.773239,0.142555,0.135230,0.281164
4,(WOODEN FRAME ANTIQUE WHITE ),(WOODEN PICTURE FRAME WHITE FINISH),0.052272,0.059708,0.029299,0.560510,9.387493,1.0,0.026178,2.139505,0.942755,0.354362,0.532602,0.525608


## Lọc các luật theo ngưỡng support/ confidence / lift

## Trực quan top các luật theo Lift

## Trực quan top luật theo confidence

## Gợi ý cách đọc & sử dụng các luật kết hợp

Một số hướng khai thác luật kết hợp trong bối cảnh kinh doanh:

- Các luật có **lift cao**:
  - Thể hiện những cặp/bộ sản phẩm gắn bó chặt chẽ hơn nhiều so với ngẫu nhiên.
  - Gợi ý cho: combo sản phẩm, trưng bày trên kệ, chương trình mua kèm.

- Các luật có **confidence cao nhưng support vừa phải**:
  - Thích hợp cho chiến dịch cá nhân hoá: 
  - Khi khách đã mua `antecedent`, xác suất mua `consequent` rất cao.

- Luật có **support cao nhưng lift không quá lớn**:
  - Thể hiện các thói quen mua sắm phổ biến.
  - Giúp hiểu "giỏ hàng trung bình" của khách hàng.

Khi kết hợp với RFM ở Notebook 01, ta có thể:
- Áp dụng luật khác nhau cho nhóm khách hàng VIP / mới / có rủi ro rời bỏ.
- Thiết kế chiến dịch gợi ý sản phẩm theo phân khúc khách hàng cụ thể.


In [10]:
rules_fp.to_csv(
    "../data/processed/rules_fpgrowth.csv",
    index=False
)

print("Đã lưu luật FP-Growth:")
print(" - File: data/processed/rules_fpgrowth.csv")
print(" - Số luật:", rules_fp.shape[0])


Đã lưu luật FP-Growth:
 - File: data/processed/rules_fpgrowth.csv
 - Số luật: 218


In [11]:
import os
print(os.getcwd())


D:\Khai-Pha-Du_Lieu\shopping_cart_analysis\notebooks
